In [ ]:
# Install required packages.
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


Graph Dataset

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.loader import DataLoader
from torch_geometric.nn import (SplineConv, global_mean_pool, graclus,
                                max_pool, max_pool_x)
from torch_geometric.utils import normalized_cut

In [ ]:
transform = T.Cartesian(cat=False)
dataset = MNISTSuperpixels(root=".", transform=transform)
data = dataset
data_size = len(data)
train_dataset = data[:int(data_size * 0.8)];
test_dataset = data[int(data_size * 0.8):];

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
d = train_dataset

In [ ]:
def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(d.num_features, 32, dim=2, kernel_size=5)
        self.conv2 = SplineConv(32, 64, dim=2, kernel_size=5)
        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, d.num_classes)

    def forward(self, data):
        data.x = F.elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        weight = normalized_cut_2d(data.edge_index, data.pos)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data, transform=transform)

        data.x = F.elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        weight = normalized_cut_2d(data.edge_index, data.pos)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        x, batch = max_pool_x(cluster, data.x, data.batch)

        x = global_mean_pool(x, batch)
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return F.log_softmax(self.fc2(x), dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train(epoch):
    model.train()

    if epoch == 16:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

    if epoch == 26:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        F.nll_loss(model(data), data.y).backward()
        optimizer.step()


def test():
    model.eval()
    correct = 0

    for data in test_loader:
        data = data.to(device)
        pred = model(data).max(1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(test_dataset)

In [ ]:
for epoch in range(1, 31):
    train(epoch)
    test_acc = test()
    print(f'Epoch: {epoch:02d}, Test: {test_acc:.4f}')

Epoch: 01, Test: 0.8153
Epoch: 02, Test: 0.8590
Epoch: 03, Test: 0.8684
Epoch: 04, Test: 0.8964
Epoch: 05, Test: 0.8702
Epoch: 06, Test: 0.9207
Epoch: 07, Test: 0.9182
Epoch: 08, Test: 0.9246
Epoch: 09, Test: 0.9177
Epoch: 10, Test: 0.9251
Epoch: 11, Test: 0.9270
Epoch: 12, Test: 0.9172
Epoch: 13, Test: 0.9273
Epoch: 14, Test: 0.9308
Epoch: 15, Test: 0.9358
Epoch: 16, Test: 0.9479
Epoch: 17, Test: 0.9496
Epoch: 18, Test: 0.9506
Epoch: 19, Test: 0.9527
Epoch: 20, Test: 0.9491
Epoch: 21, Test: 0.9497
Epoch: 22, Test: 0.9511
Epoch: 23, Test: 0.9484
Epoch: 24, Test: 0.9533
Epoch: 25, Test: 0.9519
Epoch: 26, Test: 0.9514
Epoch: 27, Test: 0.9539
Epoch: 28, Test: 0.9541
Epoch: 29, Test: 0.9513
Epoch: 30, Test: 0.9534
